### 1.默认加载

In [2]:
%load_ext autoreload
%autoreload 2

try:
    from google.colab import drive
    drive.mount('/content/drive')
    import os
    GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Vgg16'
    GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
    print(os.listdir(GOOGLE_DRIVE_PATH))
    import sys
    sys.path.append(GOOGLE_DRIVE_PATH)
except:
    pass

import time, os, torch, torchvision, random, time, math
from torch import Tensor
import torchvision
import matplotlib.pyplot as plt
from imageio import imread
from PIL import Image
from torchvision.transforms import ToTensor

%matplotlib inline
plt.rcParams['figure.figsize'] = (6, 4)
plt.rcParams['font.size'] = 10
from toolset.utils import *
from toolset.data import *
from toolset.helper import *
from toolset.solver import *
from convolutional_networks import *
from fully_connected_networks import *
from toolset import *
if torch.cuda.is_available:
    print('Good to go!')
else:
    print('Please set GPU via Edit -> Notebook Settings.')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Good to go!


### 2. unfold()

`torch.Tensor.unfold()`是一个函数，用于创建一个视图，其中沿着指定维度展开的数据被组织成一个新的最后一维。换句话说，这个操作可以用于有效地提取张量的滑动窗口块，用于进一步的操作（如卷积）。

这个函数的语法是这样的：

`tensor.unfold(dimension, size, step)`

- `dimension` 是你想要展开的维度
- `size` 是滑动窗口的大小
- `step` 是滑动窗口移动的步长



In [18]:
x = torch.arange(1, 8)  # tensor([1, 2, 3, 4, 5, 6, 7])
y = x.unfold(0, 3, 1)  # 连续的滑动窗口
print(y)

tensor([[1, 2, 3],
        [2, 3, 4],
        [3, 4, 5],
        [4, 5, 6],
        [5, 6, 7]])


In [42]:
x = torch.arange(30).view(5, 6)
print(x)
H, W = x.shape
HH = WW = 2
pad = 0
stride = 2
H_out = 1 + (H + 2 * pad - HH) // stride
W_out = 1 + (W + 2 * pad - WW) // stride

print(H_out,W_out)

def get_conv_table(x, HH, WW, stride):
    #@ 针对按顺序的两维(这里是0，1)逐个unfold就可以得到要卷积的部分
    y = x.unfold(0, HH, stride).unfold(1, WW, stride)
    return y
y = get_conv_table(x, HH,WW,stride)
#print(y)
print(y.shape)
print(y)
# 输出：
# tensor([[[[1, 2],
#            [4, 5]],
#           [[2, 3],
#            [5, 6]]],
#          [[[4, 5],
#            [7, 8]],
#           [[5, 6],
#            [8, 9]]]])

tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23],
        [24, 25, 26, 27, 28, 29]])
2 3
torch.Size([2, 3, 2, 2])
tensor([[[[ 0,  1],
          [ 6,  7]],

         [[ 2,  3],
          [ 8,  9]],

         [[ 4,  5],
          [10, 11]]],


        [[[12, 13],
          [18, 19]],

         [[14, 15],
          [20, 21]],

         [[16, 17],
          [22, 23]]]])


### 3. einsum()

In [44]:
# 转置
x = torch.arange(6).reshape((2,3))
print(x)
# tensor([[0, 1, 2],
#         [3, 4, 5]])

# 使用einsum进行转置操作
y = torch.einsum('ij->ji', x)
print(y)
# tensor([[0, 3],
#         [1, 4],
#         [2, 5]])

# 最后两维转置
a = torch.randn(2,3,5,7,9)
# i = 7, j = 9
b = torch.einsum('...ij->...ji', [a])


tensor([[0, 1, 2],
        [3, 4, 5]])
tensor([[0, 3],
        [1, 4],
        [2, 5]])


In [45]:
# 矩阵乘法
x = torch.arange(6).reshape((2,3))
y = torch.arange(9).reshape((3,3))

# 使用einsum进行矩阵乘法操作
z = torch.einsum('ij,jk->ik', x, y)
print(z)
# tensor([[15, 18, 21],
#         [42, 54, 66]])


tensor([[15, 18, 21],
        [42, 54, 66]])


In [56]:
# 张量点乘求和
x = torch.arange(3)
y = torch.arange(3, 6)
print(x, y)
# 使用einsum进行点乘操作
z = torch.einsum('i,i->i', x, y)
print(z)
# tensor([ 0,  4, 10])

# 使用einsum进行点乘+求和操作
z = torch.einsum('i,i->', x, y)
print(z)
# tensor(14)

tensor([0, 1, 2]) tensor([3, 4, 5])
tensor([ 0,  4, 10])
tensor(14)


### Conv

In [103]:
x = torch.arange(2*24).view(2, 4, 6)  # 理解成双通道图像
print(x)
C, H, W = x.shape
HH = WW = 2
pad = 0
stride = 2
H_out = 1 + (H + 2 * pad - HH) // stride
W_out = 1 + (W + 2 * pad - WW) // stride

print(H_out,W_out)

def get_conv_table(x, HH, WW, stride):
    #@ 针对按顺序的两维(这里是1，2)逐个unfold就可以得到要卷积的部分
    y = x.unfold(1, HH, stride).unfold(2, WW, stride)
    return y
y = get_conv_table(x, HH,WW,stride)
#print(y)
print(y.shape)  # (N, H_out, W_out, stride, stride)
print(y)

# 对角取元素
# 双通道图像需要双通道卷积核(单个)
w = torch.tensor([
    [[1, 0],
     [0, 1]],
    [[1, 0],
     [0, 1]]
    
])
print(w.shape)

tensor([[[ 0,  1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10, 11],
         [12, 13, 14, 15, 16, 17],
         [18, 19, 20, 21, 22, 23]],

        [[24, 25, 26, 27, 28, 29],
         [30, 31, 32, 33, 34, 35],
         [36, 37, 38, 39, 40, 41],
         [42, 43, 44, 45, 46, 47]]])
2 3
torch.Size([2, 2, 3, 2, 2])
tensor([[[[[ 0,  1],
           [ 6,  7]],

          [[ 2,  3],
           [ 8,  9]],

          [[ 4,  5],
           [10, 11]]],


         [[[12, 13],
           [18, 19]],

          [[14, 15],
           [20, 21]],

          [[16, 17],
           [22, 23]]]],



        [[[[24, 25],
           [30, 31]],

          [[26, 27],
           [32, 33]],

          [[28, 29],
           [34, 35]]],


         [[[36, 37],
           [42, 43]],

          [[38, 39],
           [44, 45]],

          [[40, 41],
           [46, 47]]]]])
torch.Size([2, 2, 2])



假设你的输入张量 `x` 的形状为 `(C, H, W)`，即通道数为 `C`，高度为 `H`，宽度为 `W`。你的卷积核 `w` 的形状为 `(C, HH, WW)`，即通道数也为 `C`，高度为 `HH`，宽度为 `WW`。此处的 `y` 是 `x` 通过 `unfold` 操作后的结果，形状为 `(C, H_out, W_out, HH, WW)`，其中 `H_out` 和 `W_out` 分别是输出的高度和宽度。

下面的 `einsum` 表达式 `'ChwIJ,CIJ->hw'` 可以表示为以下数学公式：

$$out_{hw}=\sum_{C,\text{堆叠相加}}{\sum_{I=0}^{HH-1}{\sum_{J=0}^{WW-1}{y_{ChwIJ}\cdot}}}w_{CIJ}$$

其中，`h` 和 `w` 是输出张量的高度和宽度的索引，`C` 是通道的索引，`I` 和 `J` 是卷积核的高度和宽度的索引。这个表达式的含义是，对于输出张量的每一个位置 `(h, w)`，我们遍历输入的所有通道和卷积核的所有位置，将输入的对应部分和卷积核的元素相乘，然后将所有结果相加，得到输出张量的该位置的值。

这就是使用 `einsum` 进行卷积运算的数学公式表示。你可以看到，这个表达式就是卷积运算的定义：将输入的每一个局部窗口和卷积核进行对应元素的乘积和操作，得到输出的对应位置的值。

In [105]:
# h, w是输出维度，I，J是卷积核大小
out = torch.einsum('ChwIJ,CIJ->hw', y, w)
print(out)
# tensor([[ 0+7+24+31,2+9+26+33,  78], 
#         [110, 118, 126]])

tensor([[ 62,  70,  78],
        [110, 118, 126]])



关于 `'NChwIJ,FCIJ->NFhw'` 这个表达式的数学公式如下：
$$
\text{out}_{NFhw} = \sum_{C} \sum_{I=0}^{HH-1} \sum_{J=0}^{WW-1} x_{NChwIJ} \cdot w_{FCIJ}
$$


In [115]:

class ConvMy(object):
    @staticmethod
    def forward(x:Tensor, w:Tensor, b:Tensor, conv_param: dict):
        """
        卷积层前向传播的高速实现。
        对输入张量执行2d卷积的函数。
        Args:
            x: (Tensor): 输入张量，形状为 (N, C, H, W)，其中N为批大小，C为通道数，H为图像高度，W为图像宽度。
            w: (Tensor): 卷积层的权重（即“滤波器”），形状为 (F, C, I, J)，其中F为滤波器数量，I,J代表滤波器的高度和宽度。 
                这里不采用HH，WW是为了配合einsum下标。
            b: (Tensor): 每个卷积核的偏置项。这是一个长度为F的一维张量(F,)。
            conv_param (dict): 一个具有两个键'stride'和'pad'的字典，表示在卷积操作中要使用的步长和填充。
        Return:
            Tensor: 卷积操作的输出。
            Tuple: 计算过程的cache元组，(x, w, b, conv_param)。
        """
        stride, pad = conv_param['stride'], conv_param['pad']
        N, C, H, W = x.shape
        F, C, I, J = w.shape  # I,J为卷积核大小
        # 填充
        if pad != 0:
            x_padded = torch.zeros(N, C, H + 2 * pad, W + 2 * pad, dtype=x.dtype, device=x.device)
            x_padded[:, :, pad:-pad, pad:-pad] = x
        else:
            x_padded = x
            
        # 不需要手动计算H_out，W_out，下面einsum中用h，w代替
        x_padded = x_padded.unfold(2, I, stride).unfold(3, J, stride)  # 展开成为卷积的形式
        # 'NChwIJ,FCIJ->NFhw'等式表示我们沿着高度和宽度维度（IJ）将输入窗口和滤波器进行点积，并沿输入通道（C）求和。
        out = torch.einsum('NChwIJ,FCIJ->NFhw', x_padded, w)  # (N,F,H_out,W_out)
        out += b.reshape(1, -1, 1, 1)
        return out, (x_padded, w, b, conv_param)
    
    
    @staticmethod 
    def backward(dout:Tensor, cache:Tuple):
        """
        卷积层反向传播的高速实现。
        输入：
        - dout：上游导数。 形状为 (N, F, H_out, W_out) 或者 (N, F, h, w)
        - cache：与conv_forward_naive中的(x, w, b, conv_param)相同的元组。

        返回一个元组：
        - dx：相对于x的梯度
        - dw：相对于w的梯度
        - db：相对于b的梯度
        """
        # @  修正该函数，也许你需要dive into einsum
        # @ https://zhuanlan.zhihu.com/p/361209187
        x_padded, w, b, conv_param = cache
        stride, pad = conv_param['stride'], conv_param['pad']
        db = dout.sum(dim=(0,2,3))  # (F,)
        dw = torch.einsum('NChwIJ,FCIJ->FCIJ', x_padded, dout)  # (F,C,I,J)
        w_flip = w.flip([2, 3])  # 沿着I和J轴进行翻转
        dx_padded = torch.einsum('Nfhw,FCIJ->NChwIJ', dout, w_flip)
        
        # 如果进行了padding，需要将dx进行切割，去掉padding部分
        if pad != 0:
            dx = dx_padded[:, :, pad:-pad, pad:-pad]
        else:
            dx = dx_padded
        return dx, dw, db

        
        
        
        
        
    

In [116]:
# Rel errors should be around e-11 or less
# time.time精度不够会造成除0问题，把所有的time.time换成了time.perf_counter()
from convolutional_networks import Conv, FastConv

reset_seed(0)
x = torch.randn(10, 3, 31, 31, dtype=torch.float64, device='cuda')
w = torch.randn(25, 3, 3, 3, dtype=torch.float64, device='cuda')
b = torch.randn(25, dtype=torch.float64, device='cuda')
dout = torch.randn(10, 25, 16, 16, dtype=torch.float64, device='cuda')
x_cuda, w_cuda, b_cuda, dout_cuda = x.to('cuda'), w.to('cuda'), b.to('cuda'), dout.to('cuda')
conv_param = {'stride': 2, 'pad': 1}

t0 = time.perf_counter()
out_naive, cache_naive = Conv.forward(x, w, b, conv_param)
# t1 = time.perf_counter()
# out_fast, cache_fast = FastConv.forward(x, w, b, conv_param)
# t2 = time.perf_counter()
# out_fast_cuda, cache_fast_cuda = FastConv.forward(x_cuda, w_cuda, b_cuda, conv_param)
# t3 = time.perf_counter()
# out_my, cache_my = ConvMy.forward(x, w, b, conv_param)
# t4 = time.perf_counter()

# print('Testing FastConv.forward:')
# print('Naive: %fs' % (t1 - t0))
# print('Fast: %fs' % (t2 - t1))
# print('Fast CUDA: %fs' % (t3 - t2))
# print(f'My Fast:{t4-t3:.6f}s')
# print('Speedup: %fx' % ((t1 - t0) / (t2 - t1)))
# print('Speedup CUDA: %fx' % ((t1 - t0) / (t3 - t2)))
# print('Difference: ', grad.rel_error(out_naive, out_fast))
# print('Difference CUDA: ', grad.rel_error(out_naive, out_fast_cuda.to(out_naive.device)))
# print('Difference My Fast: ', grad.rel_error(out_naive, out_my))


t0 = time.perf_counter()
dx_naive, dw_naive, db_naive = Conv.backward(dout, cache_naive)
t1 = time.perf_counter()
dx_fast, dw_fast, db_fast = FastConv.backward(dout, cache_fast)
t2 = time.perf_counter()
dx_fast_cuda, dw_fast_cuda, db_fast_cuda = FastConv.backward(dout_cuda, cache_fast_cuda)
t3 = time.perf_counter()
dx_my, dw_my, db_my = ConvMy.backward(dout_cuda, cache_my)
t4 = time.perf_counter()
print('\nTesting FastConv.backward:')
print('Naive: %fs' % (t1 - t0))
print('Fast: %fs' % (t2 - t1))
print('Fast CUDA: %fs' % (t3 - t2))
print(f'My Fast:{t4-t3:.6f}s')
print('Speedup: %fx' % ((t1 - t0) / (t2 - t1)))
print('Speedup CUDA: %fx' % ((t1 - t0) / (t3 - t2)))
print('dx difference: ', grad.rel_error(dx_naive, dx_fast))
print('dw difference: ', grad.rel_error(dw_naive, dw_fast))
print('db difference: ', grad.rel_error(db_naive, db_fast))
print('dx difference CUDA: ', grad.rel_error(dx_naive, dx_fast_cuda.to(dx_naive.device)))
print('dw difference CUDA: ', grad.rel_error(dw_naive, dw_fast_cuda.to(dw_naive.device)))
print('db difference CUDA: ', grad.rel_error(db_naive, db_fast_cuda.to(db_naive.device)))

print('dx difference My: ', grad.rel_error(dx_naive, dx_my.to(dx_naive.device)))
print('dw difference My: ', grad.rel_error(dw_naive, dw_my.to(dw_naive.device)))
print('db difference My: ', grad.rel_error(db_naive, db_my.to(db_naive.device)))

RuntimeError: einsum(): operands do not broadcast with remapped shapes [original->remapped]: [10, 3, 16, 16, 3, 3]->[1, 3, 3, 3, 10, 16, 16] [10, 25, 16, 16]->[10, 25, 16, 16, 1, 1, 1]